# Surface Classifier

Quantifying the surface characteristics of an organ is essential to understanding how it interacts with other organs and defining its function. This tool will look at one of your image stacks and calculate surface statistics (more information below) and also export an `.obj` file. You can print this `.obj` file out in the maker space if you would like. Possible insights that could be gained from this tool include:
- 3D shape
- Surface texture
- Unique surface features

<div style="display: flex; justify-content: space-around;">
    <img src="https://raw.githubusercontent.com/agadin/QP2_big_data_project_tools/refs/heads/main/img/surface_rotate1.gif" alt="Surface Rotate 1" style="width: 45%;">
    <img src="https://raw.githubusercontent.com/agadin/QP2_big_data_project_tools/refs/heads/main/img/surface_rotate2.gif" alt="Surface Rotate 2" style="width: 45%;">
</div>

### Outputs:
1. **3D Mesh File**:
   - A `.obj` file representing the 3D reconstructed mesh.
   - Contains vertices and faces to represent the 3D structure.

2. **Surface Properties**:
   The script analyzes the generated `.obj` file and outputs the following metrics:
   - **Surface Area (mm²)**: Total surface area of the mesh.
   - **Volume (mL)**: The internal volume enclosed by the mesh.
   - **Bounding Box Dimensions (mm)**: The dimensions of the axis-aligned bounding box (`x`, `y`, and `z`).
   - **Number of Triangles**: The count of triangular faces in the mesh.
   - **Resolution Metrics**:
     - Average edge length (mm).
     - Maximum edge length (mm).
     - Minimum edge length (mm).
   - **Defect Detection**:
     - Euler Number: A topological measure for connectivity.
     - Non-Manifold Edges: The count of edges that belong to more than two faces, indicating defects.

### Workflow:
1. **Load Images**: Reads all 2D slices from the specified folder and processes them into a 3D binary stack.
2. **Reconstruct Mesh**: Converts the binary stack into a 3D mesh using the marching cubes algorithm.
3. **Save Mesh**: Exports the reconstructed mesh as a `.obj` file.
4. **Analyze Mesh**: Calculates and prints various properties of the mesh for quantifying quality and resolution.

# Upload the image stack
Upload and select the folder containing the image stack you want to analyze. The image stack should be in a folder containing the image files (e.g. MRI_4, CT_1, etc.).

In [4]:
!pip install numpy opencv-python trimesh scikit-image ipython ipywidgets

import os
import ipywidgets as widgets
from IPython.display import display

current_path = os.getcwd()

path_display = widgets.Text(
    value=current_path,
    description='Path:',
    layout=widgets.Layout(width='800px')
)

output = widgets.Output()

def display_directories(path):
    """Display only directories in the given path as clickable rectangles."""
    items = [item for item in os.listdir(path) if os.path.isdir(os.path.join(path, item))]
    buttons = []

    for item in items:
        item_path = os.path.join(path, item)
        button = widgets.Button(
            description=item,
            layout=widgets.Layout(width='auto', height='30px')
        )
        button.style.button_color = '#007bff'
        button.on_click(lambda b, p=item_path: handle_directory_click(p))
        buttons.append(button)

    return widgets.VBox(buttons)

def count_png_files(path):
    """Count the number of .png files in the given directory."""
    return len([f for f in os.listdir(path) if f.endswith('.png')])

def handle_directory_click(directory_path):
    """Handle clicks on directories."""
    global current_path  # Make current_path accessible globally
    current_path = directory_path  # Update global variable
    path_display.value = current_path
    refresh_directory_view(current_path)

    # Check for .png files and display a warning if there are less than two
    png_count = count_png_files(directory_path)
    output.clear_output()
    with output:
        if png_count <= 1:
            print(f"Warning: The directory '{os.path.basename(directory_path)}' contains {png_count} .png file(s).")
        else:
            print(f"The directory '{os.path.basename(directory_path)}' contains enough ({png_count}) .png file(s).")

def navigate_to_parent_directory(_):
    """Navigate to the parent directory."""
    global current_path  # Make current_path accessible globally
    parent_path = os.path.dirname(current_path)
    current_path = parent_path  # Update global variable
    path_display.value = current_path
    refresh_directory_view(current_path)

def refresh_directory_view(path):
    """Refresh the directory view."""
    directory_view.children = [display_directories(path)]

# Button to go to the parent directory
parent_button = widgets.Button(
    description='Go to Parent Directory',
    layout=widgets.Layout(width='auto', height='30px')
)
parent_button.style.button_color = '#007bff'
parent_button.on_click(navigate_to_parent_directory)

# Initialize the directory view
directory_view = widgets.VBox(children=[display_directories(current_path)])

# Layout the widgets
navigation_box = widgets.VBox([path_display, parent_button, directory_view, output])
display(navigation_box)

# If the current path stops updating, run this cell again


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


# Calculate the surface properties
Run the following cell to calculate the surface properties of the mesh.

In [5]:
import os
import numpy as np
import cv2
import trimesh
from skimage import measure
from IPython.display import display
import trimesh.viewer.notebook

def load_image_stack(folder_path):
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.tif'))])
    stack = []
    for image_file in image_files:
        image = cv2.imread(os.path.join(folder_path, image_file), cv2.IMREAD_GRAYSCALE)
        if image is not None:
            stack.append(image)
    return np.array(stack)

def create_mesh_from_stack(image_stack, folder_path):
    binary_stack = (image_stack > 0).astype(np.uint8)
    print(folder_path)
    spacing = [4, 10 / 17.53, 10 / 17.53] if "CT" in folder_path else [1, 10 / 17.53, 10 / 17.53]
    print(spacing)
    verts, faces, _, _ = measure.marching_cubes(binary_stack, level=0.5, spacing=spacing)
    mesh = trimesh.Trimesh(vertices=verts, faces=faces)
    return mesh

def save_mesh(mesh, output_path):
    if os.path.exists(output_path):
        os.remove(output_path)
    mesh.export(output_path)

def load_mesh_from_obj(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file {file_path} does not exist.")
    return trimesh.load(file_path)

def analyze_surface_properties_from_obj(obj_path):
    mesh = load_mesh_from_obj(obj_path)

    volume_ml = abs(mesh.volume) / 1000

    bounding_box = mesh.bounds
    bounding_box_dims = {
        'x_dim_mm': bounding_box[1][0] - bounding_box[0][0],
        'y_dim_mm': bounding_box[1][1] - bounding_box[0][1],
        'z_dim_mm': bounding_box[1][2] - bounding_box[0][2]
    }

    avg_edge_length = np.mean(mesh.edges_unique_length)
    max_edge_length = np.max(mesh.edges_unique_length)
    min_edge_length = np.min(mesh.edges_unique_length)

    euler_number = mesh.euler_number
    non_manifold_edges = mesh.edges_unique.shape[0] - mesh.edges.shape[0]

    properties = {
        'surface_area_mm2': mesh.area,  # Surface area in square mm
        'volume_ml': volume_ml,  # Volume in mL
        'bounding_box_dims_mm': bounding_box_dims,  # Bounding box dimensions in mm
        'num_triangles': len(mesh.faces),  # Number of triangles in the mesh
        'avg_edge_length_mm': avg_edge_length,  # Average edge length in mm
        'max_edge_length_mm': max_edge_length,  # Maximum edge length in mm
        'min_edge_length_mm': min_edge_length,  # Minimum edge length in mm
        'euler_number': euler_number,  # Topological measure
        'non_manifold_edges': non_manifold_edges  # Number of non-manifold edges
    }
    return properties

def main():
    folder_path = current_path
    output_path = "output_mesh.obj" # Change this to your desired output path (optional)

    image_stack = load_image_stack(folder_path)

    mesh = create_mesh_from_stack(image_stack, folder_path)
    save_mesh(mesh, output_path)
    print(f"Mesh saved to {output_path}")

    properties = analyze_surface_properties_from_obj(output_path)
    print("Surface Properties from OBJ:", properties)

    scene = trimesh.Scene(mesh)
    display(scene.show())

if __name__ == "__main__":
    main()

/Users/alexandergadin/PycharmProjects/QP2_Big_data_project_tools/Group_3/MRI_2
[1, 0.5704506560182544, 0.5704506560182544]
Mesh saved to output_mesh.obj
Surface Properties from OBJ: {'surface_area_mm2': np.float64(16247.764224113007), 'volume_ml': np.float64(41.85257294631998), 'bounding_box_dims_mm': {'x_dim_mm': np.float64(75.0), 'y_dim_mm': np.float64(38.220193949999995), 'z_dim_mm': np.float64(247.00513406)}, 'num_triangles': 83472, 'avg_edge_length_mm': np.float64(0.738764958345222), 'max_edge_length_mm': np.float64(1.1512662400567657), 'min_edge_length_mm': np.float64(0.40336951587619896), 'euler_number': 2, 'non_manifold_edges': -125208}
